In [1]:
import pandas as pd
import os,glob

In [2]:
def Create_Cost_and_Strategy_file(Cost_file_path,new_dir):
    cost_xls = pd.ExcelFile(Cost_file_path)
    sheets_list = cost_xls.sheet_names

    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
    counter=1
    for sheet in sheets_list:
        cost_df = pd.read_excel(Cost_file_path, sheet_name=sheet, header=1)
        cost_df_header0 = pd.read_excel(Cost_file_path, sheet_name=sheet, header=0)
        
        rd_name_inFile = cost_df_header0.columns[0]
        # print(rd_name_inFile)
        rd_name_inFile = ' '.join(rd_name_inFile.split())
        chars_to_remove = '[<>:"/\\|?*%-_^!.]'
        translation_table = str.maketrans('', '', chars_to_remove)

        # Translate the text using the translation table
        rd_name_inFile = rd_name_inFile.translate(translation_table)
        # print(cost_df)
        S_no = cost_df['S/No']

        length_in_meters = cost_df['Length']

        maintenance_strategy = cost_df['Maintenance Strategy'].ffill()
        # print(' Data type:',type(cost_df['Unit Cost'].ffill()[1]),' Value: ',cost_df['Unit Cost'].ffill()[1])        
        unit_cost = cost_df['Unit Cost'].ffill()

        # Apply rounding only if the value is numeric
        unit_cost = unit_cost.apply(lambda x: round(float(x), 2) if pd.api.types.is_numeric_dtype(type(x)) else x)

        
        
        # if not 'Maintenance Strategy' in maintenance_strategy:
        #     print(rd_name_inFile)
        # print(S_no)
        df = pd.DataFrame({
            'S/No': S_no,
            'Length': length_in_meters,
            'Maintenance Strategy': maintenance_strategy,
            'Unit Cost': unit_cost
        })
        
        total_index = df[df['S/No'] == 'Total'].index

        if not total_index.empty:
            last_valid_index = total_index[0]
            filtered_df = df.loc[:last_valid_index-1]
        else:
            filtered_df = df

        filename = os.path.join(new_dir, f"{rd_name_inFile}_BOQ.csv")
        filtered_df.to_csv(filename, index=False)
        
        print(f"{counter} >-> {os.path.basename(filename)}")
        counter+=1
        # break
    print("************* DONE *************")

In [3]:
Cost_file_path= glob.glob('Data'+os.sep+'C&S related'+os.sep+'*.xlsx')[0]
csv_target_dir = 'Data'+os.sep+'C&S related'+os.sep+'csv files'

# Cost_file_path = 'Data'+os.sep+'C&S related'+os.sep+boq_file_name
Create_Cost_and_Strategy_file(Cost_file_path,csv_target_dir)

1 >-> Babozai Collector road I_BOQ.csv
2 >-> Babozai Collector road II_BOQ.csv
3 >-> Baghicha Dheri to Gllla Kandaro Garyala Charguli Road_BOQ.csv
4 >-> Chamdheri Collector road_BOQ.csv
5 >-> Dalasa Fatima Kuroona Road_BOQ.csv
6 >-> Ghala Dher Nawa killi Road_BOQ.csv
7 >-> Ghundo Chowk to pepal Kohi Burmol Qazi Baba_BOQ.csv
8 >-> Gulmera Muqbara Shubla Road_BOQ.csv
9 >-> Gulmera to Sajan Killi_BOQ.csv
10 >-> Gulsher Abad Saloo Dher road_BOQ.csv
11 >-> Hamza Kot Palo Dheri Sawaldher Road_BOQ.csv
12 >-> Husai Piran Kot Gujrat Road_BOQ.csv
13 >-> Kot Hathian Road_BOQ.csv
14 >-> Kudinaka Road_BOQ.csv
15 >-> Lundkhwar Badrago Hisar Darra Road_BOQ.csv
16 >-> MPremixhi Branch to Kharki_BOQ.csv
17 >-> Maday Baba Pir Saddi Qutab Garh Road_BOQ.csv
18 >-> Mardan Bakhshali Road_BOQ.csv
19 >-> Mardan Katlang Main Khan Sangao Kingargalli road_BOQ.csv
20 >-> Mardan Mayar Sowaryan Toru Marhatti Road_BOQ.csv
21 >-> Mill By Pass Road upto Gujar garhi Charsadda Chowk to GGarhi Side A_BOQ.csv
22 >-> Mill 

In [4]:
files = glob.glob(csv_target_dir+os.sep+'*.csv')

valid_strategies = [
    'PRESERVATIVE MAINTENANCE', 
    'ROUTINE MAINTENANCE', 
    'RESURFACING WITH DST', 
    'RESURFACING WITH SAND ASPHALT', 
    'FUNCTIONAL OVERLAY', 
    'STRUCTURAL OVERLAY (RECONDITIONING)', 
    'STRUCTURAL OVERLAY (REHABILITATION)', 
    'STRUCTURAL OVERLAY REHABILITATION (WITHOUT RAISING)',
    'NEW CONSTRUCTION FOR KACHA ROAD']

for file in files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        
        # Find rows where the Maintenance Strategy is not in the valid list
        df['Maintenance Strategy'] = df['Maintenance Strategy'].str.upper()
        invalid_strategies = df[~df['Maintenance Strategy'].isin(valid_strategies)]
        
        if not invalid_strategies.empty:
            print(f"Invalid strategies found in {file}:")
            print(invalid_strategies[['S/No', 'Maintenance Strategy']])
            print()
    else:
        print(f"File {file} does not exist.")
